In [13]:
from gurobipy import Model, GRB
import numpy as np
import math

In [16]:
current_inventory = {'1': 300,
                    '2': 300,
                    '3': 120,
                    '4': 120,
                    '5': 70,
                    '6': 120,
                    '7': 50,
                    '8': 0,
                    '9': 1}

In [17]:
current_day = 1
num_stores = 9

In [28]:
daily_demands = {'1': [(83,5.8), (15,7), (20,4), (25,6), (10,6), (83,5.8), (20,4)],
                '2': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '3': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '4': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '5': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '6': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '7': [(20,5), (15,7), (20,4), (25,6), (10,6), (8,5), (20,4)],
                '8': [(20,5), (15,7), (20,4), (25,6), (10,6), (6,2), (20,4)],
                '9': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)]}

In [19]:
costs = {'stockout': 30,
        'holding': 1,
        'transshipment': 1}

In [20]:
def convolution(store_id, start_day):
    
    store_data = daily_demands[store_id]
    future_data = store_data[start_day:]
    mean_values = [mean for mean, _ in future_data]
    std_dev_values = [std_dev for _, std_dev in future_data]
    convoluted_mean = sum(mean_values)
    convoluted_st_dev = math.sqrt(sum(x**2 for x in std_dev_values))
    return convoluted_mean, convoluted_st_dev

In [29]:
convolution('1', current_day)

(173, 13.661625086350451)

In [30]:
ratio = (costs['stockout'] - costs['transshipment']) / (costs['stockout'] + costs['holding'] - costs['transshipment'])
ratio

0.9666666666666667

In [31]:
def update_inventory(minus):

    for store in current_inventory:
        current_inventory[store] -= minus.get(store, 0)

In [10]:
todays_sales = {'1': 10,
                    '2': 50,
                    '3': 20,
                    '4': 20,
                    '5': 20,
                    '6': 20,
                    '7': 20,
                    '8': 20,
                    '9': 20}

In [11]:
update_inventory(todays_sales)

In [32]:
current_inventory

{'1': 300,
 '2': 300,
 '3': 120,
 '4': 120,
 '5': 70,
 '6': 120,
 '7': 50,
 '8': 0,
 '9': 1}

In [33]:
shipper_cand = []
def shipper_candidates(current_inventory, current_day):
    for i in current_inventory.keys():
        mean, st_dev = convolution(i, current_day)
        critical_point = mean + (ratio*st_dev)
        if critical_point < current_inventory[i]:
            print(f"for store {i} critical point, current inventory: {critical_point, current_inventory[i]}")
            shipper_cand.append(i)
    return shipper_cand

In [34]:
shipper_candidates(current_inventory, current_day)

for store 1 critical point, current inventory: (186.2062375834721, 300)
for store 2 critical point, current inventory: (113.7389147234335, 300)
for store 3 critical point, current inventory: (113.7389147234335, 120)
for store 4 critical point, current inventory: (113.7389147234335, 120)
for store 6 critical point, current inventory: (113.7389147234335, 120)


['1', '2', '3', '4', '6']

In [35]:
receiving_cand = []
def receiving_candidates(current_inventory, current_day):
    for i in current_inventory.keys():
        mean, st_dev = convolution(i, current_day)
        critical_point = mean - (ratio*st_dev)
        if critical_point > current_inventory[i]:
            print(f"for store {i} critical point, current inventory: {critical_point, current_inventory[i]}")
            receiving_cand.append(i)
    return receiving_cand

In [36]:
receiving_candidates(current_inventory, current_day)

for store 5 critical point, current inventory: (86.2610852765665, 70)
for store 7 critical point, current inventory: (85.10305807134455, 50)
for store 8 critical point, current inventory: (83.88770138339639, 0)
for store 9 critical point, current inventory: (86.2610852765665, 1)


['5', '7', '8', '9']

In [37]:
model = Model("Lateral Transshipments")

Restricted license - for non-production use only - expires 2025-11-24


In [38]:
results = []

In [41]:
for shipper in shipper_cand:
    shipper_mean, shipper_st_dev = convolution(shipper, current_day)
    shipper_inventory = current_inventory[shipper]
    for receiver in receiving_cand:
        receiver_mean, receiver_st_dev = convolution(receiver, current_day)
        receiver_inventory = current_inventory[receiver]
        
        x = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="amount")
        
        s_shipper = model.addVar(lb=0, name="s_shipper")
        s_receiver = model.addVar(lb=0, name="s_receiver")

        model.addConstr(s_shipper >= shipper_mean - (shipper_inventory + x))
        model.addConstr(s_receiver >= receiver_mean - (receiver_inventory - x))

        # Amaç fonksiyonunu güncelle
        model.setObjective(30 * s_shipper + 30 * s_receiver + x, GRB.MINIMIZE)

        
        model.optimize()
        
        optimized_x = x.X
        minimized_cost = model.ObjVal 
                

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 4 columns and 4 nonzeros
Model fingerprint: 0x4199db0d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+02]
Presolve removed 2 rows and 4 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  9.000000000e+02
0.0 900.0
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 

  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   9.600000e+01   0.000000e+00      0s
       1    2.8800000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.880000000e+03
0.0 2880.0
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 24 rows, 37 columns and 48 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf. 